In [1]:
import numpy as np
import pandas as pd  
import torch

data = np.load('C:/Users/Hoda/A - Uni/thesis/AGCRN_GSL-init/data/PEMS04/pems04.npz')['data'][:, :, 0]  


In [2]:
import sys
sys.argv=['']
del sys

In [3]:
import os
import sys
__file__ = 'C:/Users/Hoda/A - Uni/AGCRN/data/PEMS04.npz'
file_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
print(file_dir)
sys.path.append(file_dir)

C:\Users\Hoda\A - Uni\AGCRN


In [1]:
from lib.dataloader import *
import numpy as np

# Step 1: Generate the adjacency matrix and embeddings
dataset_name = "PEMSD4"
val_ratio = 0.2
test_ratio = 0.2
prediction_length = 1  # Set prediction length
lambda1 = 0.02
embedding_dim = 10  # Set embedding dimensions

adj_matrix, embeddings = get_train_adj_matrix_and_embeddings(
    dataset_name, val_ratio, test_ratio, prediction_length, lambda1, embedding_dim)

# Step 2: Print adjacency matrix shape
print(f"Adjacency Matrix Shape: {adj_matrix.shape}")

# Step 3: Save the adjacency matrix and embeddings
adj_matrix_path = "adj_matrix.npy"
embeddings_path = "embeddings.pt"

np.save(adj_matrix_path, adj_matrix)
torch.save(embeddings, embeddings_path)

print(f"Adjacency matrix saved to {adj_matrix_path}")
print(f"Embeddings saved to {embeddings_path}")


Load PEMSD4 Dataset shaped:  (16992, 307, 1) 919.0 0.0 211.7007794815878 180.0
Normalize the dataset by MinMax01 Normalization


  0%|          | 0/180000.0 [00:00<?, ?it/s]

Generated Adjacency Matrix Shape: (307, 307)
NMF Embedding shape: torch.Size([307, 10])
Adjacency Matrix Shape: (307, 307)
Adjacency matrix saved to adj_matrix.npy
Embeddings saved to embeddings.pt


In [5]:
# Step 3: Generate NMF-based embeddings
precomputed_embeddings = torch.load('C:/Users/Hoda/A - Uni/thesis/AGCRN_GSL-init/embeddings.pt')

# Step 4: Print or use embeddings
print(f"Precomputed Embeddings Shape: {precomputed_embeddings.shape}")


Precomputed Embeddings Shape: torch.Size([307, 10])


In [4]:
import torch
# Define the path to the adjacency matrix
adj_file_name = "C:/Users/Hoda/A - Uni/thesis/est_adj_agcrn/E_init_NMF_nrmlze.npy"

# Check if the file exists and load it
if os.path.exists(adj_file_name):
    # Load the matrix from the file
    precomputed_embeddings = torch.tensor(np.load(adj_file_name), dtype=torch.float32)
    print(f"Loaded precomputed embeddings from {adj_file_name} with shape {precomputed_embeddings.shape}")
else:
    precomputed_embeddings = None
    print(f"Embedding file not found at {adj_file_name}; using random initialization.")


Loaded precomputed embeddings from C:/Users/Hoda/A - Uni/thesis/est_adj_agcrn/E_init_NMF_nrmlze.npy with shape torch.Size([307, 10])


In [5]:
import torch
import numpy as np
import torch.nn as nn
import argparse
import configparser
from datetime import datetime
from model.AGCRN import AGCRN as Network
from model.BasicTrainer import Trainer
from lib.TrainInits import init_seed
from lib.dataloader import get_dataloader
from lib.TrainInits import print_model_parameters


#*************************************************************************#
Mode = 'Train'
DEBUG = 'True'
DATASET = 'PEMSD4'      #PEMSD4 or PEMSD8
DEVICE = 'cuda:0'
MODEL = 'AGCRN'

#get configuration
config_file = 'model/PEMSD4_AGCRN.conf'  # Assuming the file is in the same directory as your script
config = configparser.ConfigParser()
config.read(config_file)

from lib.metrics import MAE_torch
def masked_mae_loss(scaler, mask_value):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        mae = MAE_torch(pred=preds, true=labels, mask_value=mask_value)
        return mae
    return loss

#parser
args = argparse.ArgumentParser(description='arguments')
args.add_argument('--dataset', default=DATASET, type=str)
args.add_argument('--mode', default=Mode, type=str)
args.add_argument('--device', default=DEVICE, type=str, help='indices of GPUs')
args.add_argument('--debug', default=DEBUG, type=eval)
args.add_argument('--model', default=MODEL, type=str)
args.add_argument('--cuda', default=True, type=bool)
#data
args.add_argument('--val_ratio', default=config['data']['val_ratio'], type=float)
args.add_argument('--test_ratio', default=config['data']['test_ratio'], type=float)
args.add_argument('--lag', default=config['data']['lag'], type=int)
args.add_argument('--horizon', default=config['data']['horizon'], type=int)
args.add_argument('--num_nodes', default=config['data']['num_nodes'], type=int)
args.add_argument('--tod', default=config['data']['tod'], type=eval)
args.add_argument('--normalizer', default=config['data']['normalizer'], type=str)
args.add_argument('--column_wise', default=config['data']['column_wise'], type=eval)
args.add_argument('--default_graph', default=config['data']['default_graph'], type=eval)
#model
args.add_argument('--input_dim', default=config['model']['input_dim'], type=int)
args.add_argument('--output_dim', default=config['model']['output_dim'], type=int)
args.add_argument('--embed_dim', default=config['model']['embed_dim'], type=int)
args.add_argument('--rnn_units', default=config['model']['rnn_units'], type=int)
args.add_argument('--num_layers', default=config['model']['num_layers'], type=int)
args.add_argument('--cheb_k', default=config['model']['cheb_order'], type=int)
#train
args.add_argument('--loss_func', default=config['train']['loss_func'], type=str)
args.add_argument('--seed', default=config['train']['seed'], type=int)
args.add_argument('--batch_size', default=config['train']['batch_size'], type=int)
args.add_argument('--epochs', default=config['train']['epochs'], type=int)
args.add_argument('--lr_init', default=config['train']['lr_init'], type=float)
args.add_argument('--lr_decay', default=config['train']['lr_decay'], type=eval)
args.add_argument('--lr_decay_rate', default=config['train']['lr_decay_rate'], type=float)
args.add_argument('--lr_decay_step', default=config['train']['lr_decay_step'], type=str)
args.add_argument('--early_stop', default=config['train']['early_stop'], type=eval)
args.add_argument('--early_stop_patience', default=config['train']['early_stop_patience'], type=int)
args.add_argument('--grad_norm', default=config['train']['grad_norm'], type=eval)
args.add_argument('--max_grad_norm', default=config['train']['max_grad_norm'], type=int)
args.add_argument('--teacher_forcing', default=False, type=bool)
#args.add_argument('--tf_decay_steps', default=2000, type=int, help='teacher forcing decay steps')
args.add_argument('--real_value', default=config['train']['real_value'], type=eval, help = 'use real value for loss calculation')
#test
args.add_argument('--mae_thresh', default=config['test']['mae_thresh'], type=eval)
args.add_argument('--mape_thresh', default=config['test']['mape_thresh'], type=float)
#log
args.add_argument('--log_dir', default='./', type=str)
args.add_argument('--log_step', default=config['log']['log_step'], type=int)
args.add_argument('--plot', default=config['log']['plot'], type=eval)
args = args.parse_args()
init_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.set_device(int(args.device[5]))
else:
    args.device = 'cpu'


#init model
# Initialize the model with precomputed embeddings
model = Network(args, precomputed_embeddings=precomputed_embeddings)
model = model.to(args.device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    else:
        nn.init.uniform_(p)
print_model_parameters(model, only_num=False)

#load dataset
train_loader, val_loader, test_loader, scaler = get_dataloader(args,
                                                               normalizer=args.normalizer,
                                                               tod=args.tod, dow=False,
                                                               weather=False, single=False)


*****************Model Parameter*****************
node_embeddings torch.Size([307, 10]) False
encoder.dcrnn_cells.0.gate.weights_pool torch.Size([10, 2, 65, 128]) True
encoder.dcrnn_cells.0.gate.bias_pool torch.Size([10, 128]) True
encoder.dcrnn_cells.0.update.weights_pool torch.Size([10, 2, 65, 64]) True
encoder.dcrnn_cells.0.update.bias_pool torch.Size([10, 64]) True
encoder.dcrnn_cells.1.gate.weights_pool torch.Size([10, 2, 128, 128]) True
encoder.dcrnn_cells.1.gate.bias_pool torch.Size([10, 128]) True
encoder.dcrnn_cells.1.update.weights_pool torch.Size([10, 2, 128, 64]) True
encoder.dcrnn_cells.1.update.bias_pool torch.Size([10, 64]) True
end_conv.weight torch.Size([12, 1, 1, 64]) True
end_conv.bias torch.Size([12]) True
Total params num: 748810
*****************Finish Parameter****************
Load PEMSD4 Dataset shaped:  (16992, 307, 1) 919.0 0.0 211.7007794815878 180.0
Train:  (10173, 12, 307, 1) (10173, 12, 307, 1)
Val:    (3375, 12, 307, 1) (3375, 12, 307, 1)
Test:   (3375, 1

In [ ]:

if args.loss_func == 'mask_mae':
    loss = masked_mae_loss(scaler, mask_value=0.0)
elif args.loss_func == 'mae':
    loss = torch.nn.L1Loss().to(args.device)
elif args.loss_func == 'mse':
    loss = torch.nn.MSELoss().to(args.device)
else:
    raise ValueError

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init, eps=1.0e-8,
                             weight_decay=0, amsgrad=False)

#learning rate decay
lr_scheduler = None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)
    #lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=64)

#config log path
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
current_dir = os.path.dirname(os.path.realpath(__file__))
log_dir = os.path.join(current_dir,'experiments', args.dataset, current_time)
args.log_dir = log_dir

#start training
trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler,
                  args, lr_scheduler=lr_scheduler)

if args.mode.lower() == 'train':
    trainer.train()
elif args.mode.lower() == 'test':
    # Load the model on CPU
    model.load_state_dict(torch.load('../pre-trained/.pth'.format(args.dataset), map_location=torch.device('cpu')))
    model.to(torch.device('cpu'))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
else:
    raise ValueError("Invalid mode specified: {}".format(args.mode))
# 

2024-12-22 09:44: Experiment log path in: C:\Users\Hoda\A - Uni\AGCRN\data\experiments\PEMSD4\20241222094435
2024-12-22 09:44: Train Epoch 1: 0/158 Loss: 184.452835
2024-12-22 09:46: Train Epoch 1: 20/158 Loss: 202.696243
2024-12-22 09:47: Train Epoch 1: 40/158 Loss: 194.430710


In [6]:
# Adjust embeddings to match expected node count
expected_nodes = args.num_nodes  # 358
current_nodes, embed_dim = precomputed_embeddings.shape  # 307, embed_dim
if current_nodes < expected_nodes:
    padding = torch.zeros(expected_nodes - current_nodes, embed_dim)  # Zero embeddings for missing nodes
    precomputed_embeddings = torch.cat((precomputed_embeddings, padding), dim=0)


In [ ]:
# learning rate 0.003---> 0.005
if args.loss_func == 'mask_mae':
    loss = masked_mae_loss(scaler, mask_value=0.0)
elif args.loss_func == 'mae':
    loss = torch.nn.L1Loss().to(args.device)
elif args.loss_func == 'mse':
    loss = torch.nn.MSELoss().to(args.device)
else:
    raise ValueError

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init, eps=1.0e-8,
                             weight_decay=0, amsgrad=False)

#learning rate decay
lr_scheduler = None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)
    #lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=64)

#config log path
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
current_dir = os.path.dirname(os.path.realpath(__file__))
log_dir = os.path.join(current_dir,'experiments', args.dataset, current_time)
args.log_dir = log_dir

#start training
trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler,
                  args, lr_scheduler=lr_scheduler)

if args.mode.lower() == 'train':
    trainer.train()
elif args.mode.lower() == 'test':
    # Load the model on CPU
    model.load_state_dict(torch.load('../pre-trained/.pth'.format(args.dataset), map_location=torch.device('cpu')))
    model.to(torch.device('cpu'))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
else:
    raise ValueError("Invalid mode specified: {}".format(args.mode))
# 

2024-12-18 07:46: Experiment log path in: C:\Users\Hoda\A - Uni\AGCRN\data\experiments\PEMSD4\20241218074645
2024-12-18 07:46: Train Epoch 1: 0/158 Loss: 184.452835
2024-12-18 07:48: Train Epoch 1: 20/158 Loss: 202.696243
2024-12-18 07:49: Train Epoch 1: 40/158 Loss: 194.430710
2024-12-18 07:51: Train Epoch 1: 60/158 Loss: 188.901917
2024-12-18 07:52: Train Epoch 1: 80/158 Loss: 145.514496
2024-12-18 07:54: Train Epoch 1: 100/158 Loss: 185.942703
2024-12-18 07:55: Train Epoch 1: 120/158 Loss: 162.186142
2024-12-18 07:56: Train Epoch 1: 140/158 Loss: 136.544693
2024-12-18 07:58: Train Epoch 1: averaged Loss: 182.000048, MAE: 182.0000, RMSE: 237.5332, MAPE: 0.9057, tf_ratio: 1.000000, Time: 680.57 seconds
2024-12-18 07:59: Validation Epoch 1: average Loss: 175.115212, MAE: 175.1152, RMSE: 231.0441, MAPE: 0.9443, Time: 93.24 seconds
2024-12-18 07:59: ********** Current best model saved!
2024-12-18 07:59: Train Epoch 2: 0/158 Loss: 167.912537
2024-12-18 08:01: Train Epoch 2: 20/158 Loss: 1

### initializing embedding
#### Freezeing the E

In [ ]:

if args.loss_func == 'mask_mae':
    loss = masked_mae_loss(scaler, mask_value=0.0)
elif args.loss_func == 'mae':
    loss = torch.nn.L1Loss().to(args.device)
elif args.loss_func == 'mse':
    loss = torch.nn.MSELoss().to(args.device)
else:
    raise ValueError

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init, eps=1.0e-8,
                             weight_decay=0, amsgrad=False)

#learning rate decay
lr_scheduler = None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)
    #lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=64)

#config log path
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
current_dir = os.path.dirname(os.path.realpath(__file__))
log_dir = os.path.join(current_dir,'experiments', args.dataset, current_time)
args.log_dir = log_dir

#start training
trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler,
                  args, lr_scheduler=lr_scheduler)

if args.mode.lower() == 'train':
    trainer.train()
elif args.mode.lower() == 'test':
    # Load the model on CPU
    model.load_state_dict(torch.load('../pre-trained/.pth'.format(args.dataset), map_location=torch.device('cpu')))
    model.to(torch.device('cpu'))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
else:
    raise ValueError("Invalid mode specified: {}".format(args.mode))
# 

2024-12-16 10:35: Experiment log path in: C:\Users\Hoda\A - Uni\AGCRN\data\experiments\PEMSD4\20241216103526
2024-12-16 10:35: Train Epoch 1: 0/158 Loss: 184.452835
2024-12-16 10:36: Train Epoch 1: 20/158 Loss: 204.920364
2024-12-16 10:38: Train Epoch 1: 40/158 Loss: 201.505905
2024-12-16 10:39: Train Epoch 1: 60/158 Loss: 196.307312
2024-12-16 10:41: Train Epoch 1: 80/158 Loss: 153.899933
2024-12-16 10:42: Train Epoch 1: 100/158 Loss: 196.483658
2024-12-16 10:44: Train Epoch 1: 120/158 Loss: 173.205887
2024-12-16 10:45: Train Epoch 1: 140/158 Loss: 146.412079
2024-12-16 10:46: **********Train Epoch 1: averaged Loss: 189.967392, tf_ratio: 1.000000
2024-12-16 10:48: **********Val Epoch 1: average Loss: 188.231514
2024-12-16 10:48: *********************************Current best model saved!
2024-12-16 10:48: Train Epoch 2: 0/158 Loss: 181.957031
2024-12-16 10:49: Train Epoch 2: 20/158 Loss: 146.381958
2024-12-16 10:51: Train Epoch 2: 40/158 Loss: 169.571121
2024-12-16 10:52: Train Epoch 2

In [ ]:
# Save metrics to JSON file at the end of training
metrics_path = os.path.join(self.args.log_dir, 'training_metrics.json')
with open(metrics_path, 'w') as f:
    json.dump(self.metrics, f, indent=4)
self.logger.info(f"Training metrics saved to {metrics_path}")


In [ ]:
import json

with open('path/to/log_dir/training_metrics.json', 'r') as f:
    metrics = json.load(f)

# Access metrics for comparison
train_loss_epoch_1 = metrics['train']['epoch_1']['loss']
val_mae_epoch_1 = metrics['val']['epoch_1']['mae']


In [ ]:
import json
import matplotlib.pyplot as plt

# Load metrics from JSON file
metrics_path = 'path/to/log_dir/training_metrics.json'
with open(metrics_path, 'r') as f:
    metrics = json.load(f)

# Extract data for plotting
epochs = range(1, len(metrics['train']) + 1)
train_loss = [metrics['train'][f'epoch_{epoch}']['loss'] for epoch in epochs]
val_loss = [metrics['val'][f'epoch_{epoch}']['loss'] for epoch in epochs]
train_mae = [metrics['train'][f'epoch_{epoch}']['mae'] for epoch in epochs]
val_mae = [metrics['val'][f'epoch_{epoch}']['mae'] for epoch in epochs]
epoch_times = [metrics['train'][f'epoch_{epoch}']['time'] for epoch in epochs]

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()

# Plot training and validation MAE
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_mae, label='Train MAE', marker='o')
plt.plot(epochs, val_mae, label='Validation MAE', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Training and Validation MAE')
plt.legend()
plt.grid()
plt.show()

# Plot epoch times
plt.figure(figsize=(10, 6))
plt.plot(epochs, epoch_times, label='Epoch Time', marker='o', color='green')
plt.xlabel('Epoch')
plt.ylabel('Time (seconds)')
plt.title('Time Per Epoch')
plt.legend()
plt.grid()
plt.show()


# E_init, SVD

In [9]:
# SVD
# with initializing embedding
#init loss function, optimizer
# train on PEMSD4
# train on traffic flow
if args.loss_func == 'mask_mae':
    loss = masked_mae_loss(scaler, mask_value=0.0)
elif args.loss_func == 'mae':
    loss = torch.nn.L1Loss().to(args.device)
elif args.loss_func == 'mse':
    loss = torch.nn.MSELoss().to(args.device)
else:
    raise ValueError

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init, eps=1.0e-8,
                             weight_decay=0, amsgrad=False)

#learning rate decay
lr_scheduler = None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)
    #lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=64)

#config log path
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
current_dir = os.path.dirname(os.path.realpath(__file__))
log_dir = os.path.join(current_dir,'experiments', args.dataset, current_time)
args.log_dir = log_dir

#start training
trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler,
                  args, lr_scheduler=lr_scheduler)

if args.mode.lower() == 'train':
    trainer.train()
elif args.mode.lower() == 'test':
    # Load the model on CPU
    model.load_state_dict(torch.load('../pre-trained/.pth'.format(args.dataset), map_location=torch.device('cpu')))
    model.to(torch.device('cpu'))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
else:
    raise ValueError("Invalid mode specified: {}".format(args.mode))


2024-12-12 11:20: Experiment log path in: C:\Users\Hoda\A - Uni\AGCRN\data\experiments\PEMSD4\20241212112010
2024-12-12 11:20: Train Epoch 1: 0/158 Loss: 184.452835
2024-12-12 11:21: Train Epoch 1: 20/158 Loss: 201.930542
2024-12-12 11:22: Train Epoch 1: 40/158 Loss: 199.674881
2024-12-12 11:24: Train Epoch 1: 60/158 Loss: 197.639404
2024-12-12 11:25: Train Epoch 1: 80/158 Loss: 156.176819
2024-12-12 11:26: Train Epoch 1: 100/158 Loss: 199.525040
2024-12-12 11:28: Train Epoch 1: 120/158 Loss: 176.501999
2024-12-12 11:29: Train Epoch 1: 140/158 Loss: 149.526260
2024-12-12 11:30: **********Train Epoch 1: averaged Loss: 191.300976, tf_ratio: 1.000000
2024-12-12 11:31: **********Val Epoch 1: average Loss: 191.946438
2024-12-12 11:31: *********************************Current best model saved!
2024-12-12 11:31: Train Epoch 2: 0/158 Loss: 185.894775
2024-12-12 11:33: Train Epoch 2: 20/158 Loss: 149.874542
2024-12-12 11:34: Train Epoch 2: 40/158 Loss: 173.394089
2024-12-12 11:36: Train Epoch 2

KeyboardInterrupt: 